## Importing relevant libraries

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
from qiskit_algorithms.utils import algorithm_globals
# from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

## Helper functions

In [ ]:
def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))

    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]

    return op_new

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

def compute_expectation_x_squared_simpson(x, y, n):
    """
    Computes the expectation value of x^2 using Simpson's rule for numerical integration.
    
    Parameters:
    x (array-like): Discrete values of x.
    y (array-like): Corresponding values of the probability density function (PDF) at x.
    
    Returns:
    float: The expectation value of x^2.
    """
    # Ensure x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)
    
    # Compute x^2
    x_squared = x**n
    
    # Check if the number of intervals is even, if not make it even by truncating the last point
    if len(x) % 2 == 0:
        x = x[:-1]
        y = y[:-1]
        x_squared = x_squared[:-1]
    
    # Compute the integral using Simpson's rule
    h = (x[-1] - x[0]) / (len(x) - 1)
    integral = y[0] * x_squared[0] + y[-1] * x_squared[-1] + \
               4 * np.sum(y[1:-1:2] * x_squared[1:-1:2]) + \
               2 * np.sum(y[2:-2:2] * x_squared[2:-2:2])
    integral *= h / 3
    
    return integral

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

# Fidelity measure 1
def get_fidelity(zeromode_classic, zeromode_vqe):
    # Function to compute the infidelity

    overlap = np.dot(np.transpose(zeromode_vqe), zeromode_classic)
    fidelity = 1 - overlap ** 2
    return fidelity

# Fidelity measure 2
def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def compute_errors(expect_classical, expect_quantum):

    error = np.abs(expect_classical - expect_quantum) / expect_classical
    return error

## VQE implementation

In [ ]:
# VQE run for a given ansatz depth and optimizer
def run_vqe_ansatz_analysis(matrix, ansatz_depth, optimizer, seed, exact_ground_state):

    # Get the Pauli-decomposed form of the operator
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))

    # Define the variational ansatz
    ansatz = RealAmplitudes(num_qubits = num_qubits, reps = ansatz_depth)

    # Set up the random initial point
    np.random.seed(seed)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)

    # Initialize the Estimator primitive
    estimator = Estimator()

    # Run computations
    @dataclass
    class VQELog:
        parameters: list
        values: list
        def update(self, count, parameters, mean, _metadata):
            self.values.append(mean)
            self.parameters.append(parameters)
            
    log = VQELog([], [])
    
    # Run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point = initial_point, callback = log.update)

    # Get the VQE results
    result = vqe.compute_minimum_eigenvalue(qub_hamiltonian)
    length = len(exact_ground_state)

    # Get the zeromode
    optimal_params = result.optimal_point
    final_circuit = ansatz.assign_parameters(optimal_params)
    vqe_statevector = Statevector.from_instruction(final_circuit)

    # Convert the quantum and classical zeromodes into 4 x 1 arrays
    exact_ground_state = np.array(exact_ground_state).reshape((len(exact_ground_state), 1))
    vqe_statevector = vqe_statevector.data.tolist()

    if len(exact_ground_state) == 6:
        vqe_statevector = vqe_statevector[:6]
        zeromode = np.array(vqe_statevector).reshape((len(exact_ground_state), 1))
    
        # Compute the fidelity measure
        fidelity_value = get_similarity(exact_ground_state, zeromode)
    
    else:
        zeromode = np.array(vqe_statevector).reshape((len(exact_ground_state), 1))
    
        # Compute the fidelity measure
        fidelity_value = get_similarity(exact_ground_state, zeromode)
    
    return zeromode, fidelity_value

## Estimating min. ansatz depth to meet the target fidelity threshold

In [ ]:
def get_min_ansatz_depth(matrix, zeromode_classic, optimizers, num_qubits, target_fidelity_threshold):
    # Get the min. ansatz depth for a set of optimizers (for a target fidelity threshold)

    # Initial depth 
    initial_depth = 1
    max_depth = 10  # Set the maximum number of depths to check

    # Initialize a dictionary to store minimum depths for each optimizer and the zeromodes
    min_depths = {}
    zeromodes = {}
    
    # Loop through each optimizer
    for optimizer in optimizers:
        optimizer_name = optimizer.__class__.__name__
        print(f"\nRunning VQE for optimizer: {optimizer_name}")
        
        current_depth = initial_depth
        converged = False  # Flag to check if convergence occurs
        
        while current_depth <= max_depth:  # Loop for up to max_depth
            # Temporary storage for the fidelity results
            fidelities = []
    
            # Perform multiple independent VQE runs to calculate average fidelity
            for run in range(10):  # Number of independent runs
                # Set a seed for a specific VQE run
                seed = run + 1
                
                # Run VQE
                zeromode, fidelity_value = run_vqe_ansatz_analysis(matrix = matrix, ansatz_depth = current_depth, \
                                                         optimizer = optimizer, seed = seed, \
                                                         exact_ground_state = zeromode_classic)
                
                # Calculate the fidelity and append the relative error 
                fidelities.append(fidelity_value)
    
            # Calculate the average fidelity and relative error over the runs
            average_fidelity = np.mean(fidelities)
            print(f"Depth {current_depth}: Average fidelity = {average_fidelity}")
    
            # Check if the average fidelity meets the threshold
            if average_fidelity >= target_fidelity_threshold:
                min_depths[optimizer_name] = current_depth
                zeromodes[optimizer_name] = zeromode
                print(f"Minimum ansatz depth for {optimizer_name} to achieve target fidelity: {current_depth}")
                print(f"Zeromode for {optimizer_name} at optimal fidelity is: {zeromode}")
                converged = True
                break  # Exit the loop if the threshold is met
    
            current_depth += 1  # Increase depth and try again
        
        # If the loop finishes and no convergence occurs, mark as "did not converge"
        if not converged:
            min_depths[optimizer_name] = "Did not converge"
            zeromodes[optimizer_name] = "Did not converge"
            print(f"{optimizer_name} did not converge within {max_depth} depths.")
    
    return min_depths, zeromodes